In [312]:
import os
import json
import pandas as pd
import traceback

# https://python.langchain.com/docs/integrations/chat/openai
from langchain.chat_models import ChatOpenAI

In [313]:
from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [314]:
APIKEY_OPENAI = os.getenv("APIKEY_OPENAI")

In [315]:
# Initialize ChatopenAI
llm = ChatOpenAI(openai_api_key=APIKEY_OPENAI, model_name="gpt-3.5-turbo",temperature=0.5)

In [316]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
from langchain.callbacks import get_openai_callback

In [317]:
# JSON response for the chatbot
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct_answer"
    },
    "5": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    }
}

In [342]:
# Template 1 for the prompt
TEMPLATE ="""
text:{text}
You are an expert MCQ maker. Give the above text, it is your job to\
create a quize of {number} multiple choice questions for {subject} in a {tone} tone\
Make sure questions are not repeated and check all question to be confirming to the text as well. \
Make sure to format your resepomse like RESPONSE_JSON below and use it as a guide.  \
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
"""


In [343]:
# Quiz generation prompt
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [344]:
# Quiz generation chain
quize_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [345]:
# Template 2 for the prompt
TEMPLATE2="""
Your are an expert English grammerian. Give a multiple choice quiz for {subject} students. \
    you need to evaluate the complexity of the question and give a complete analysis of the quiz. 
    Only use at max 50 words for the complexity analysis. \
    If the questions are not in par with the student's cognitive and analytical abilities of the student, \
    Update the questions which needs to be changed and change the tone such that it perfectly fits the student's cognitive and analytical abilities. \
    Quiz_MCQs: 
    {quiz}

    check from an expert English writer of the above quiz:
    """

In [346]:
# Question evaluation prompt
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [347]:
# Quiz evaluation chain
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [348]:
# Sequential chain for quiz generation and evaluation
generate_evaluate_chain = SequentialChain(  
    chains=[quize_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [349]:
# File path for the Text data
file_path = r"/home/pjlinux/projects/mcqgen/data.txt"

In [350]:
# Read the TEXT data
with open(file_path, "r") as file:
    TEXT = file.read()

In [351]:
# Serialize python dictionary to JSON
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}}'

In [352]:
NUMBER = 5
SUBJECT = "Production Possibility Frontier"
TONE = "difficult"

In [353]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to use the token usage tracking
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Demystifying Choices: Understanding the Production Possibility Curve
Imagine a world where you can have everything you desire. Unfortunately, reality presents limitations. In economics, the Production Possibility Curve (PPC) depicts this very concept – the trade-offs between producing two goods or services within a given timeframe and resource constraints.

Visualizing Scarcity: The Curve Explained

The PPC is a graph with two axes representing the production quantities of two goods (Good A and Good B). Each point on the curve signifies a feasible production combination achievable with the available resources (labor, capital, technology). Points inside the curve imply underutilization of resources, while points outside represent an unrealistic scenario exceeding resource capacity.

The PPC typically slopes downward from left to right, signifying the trade-off. To produce more of 

In [366]:
# Print token usage
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 1813
Prompt Tokens: 1368
Completion Tokens: 445
Total Cost (USD): $0.002942


In [390]:
response

{'text': 'Demystifying Choices: Understanding the Production Possibility Curve\nImagine a world where you can have everything you desire. Unfortunately, reality presents limitations. In economics, the Production Possibility Curve (PPC) depicts this very concept – the trade-offs between producing two goods or services within a given timeframe and resource constraints.\n\nVisualizing Scarcity: The Curve Explained\n\nThe PPC is a graph with two axes representing the production quantities of two goods (Good A and Good B). Each point on the curve signifies a feasible production combination achievable with the available resources (labor, capital, technology). Points inside the curve imply underutilization of resources, while points outside represent an unrealistic scenario exceeding resource capacity.\n\nThe PPC typically slopes downward from left to right, signifying the trade-off. To produce more of Good A, you must sacrifice some production of Good B, and vice versa. This reflects the sca

In [391]:
quiz = response.get("quiz")

In [392]:
quiz = json.loads(quiz)

In [393]:
review = response.get("review")

In [394]:
print(review)

The complexity of the questions in the quiz is suitable for students studying the Production Possibility Frontier. The questions cover key concepts such as the representation of the PPC, the significance of a downward-sloping PPC, curved PPC lines, technological advancements impact, and limitations of the PPC model. The options provided require a good understanding of the topic to choose the correct answer. The language used is clear and concise, making it accessible for students at this level.


In [395]:
# Create a table for the quiz data
quiz_table_data = []
for key, value in quiz.items():
    mcq = value.get("mcq")
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "choices": options, "correct": correct})


In [396]:
quiz_table_data

[{'MCQ': 'What does the Production Possibility Curve (PPC) represent in economics?',
  'choices': 'a : Unlimited production possibilities | b : Feasible production combinations with resource constraints | c : Unrealistic scenarios exceeding resource capacity | d : Underutilization of resources',
  'correct': 'b'},
 {'MCQ': 'What does a downward-sloping PPC signify?',
  'choices': 'a : Increasing resource capacity | b : No trade-offs between goods | c : Scarcity of resources and trade-offs | d : Efficient resource utilization',
  'correct': 'c'},
 {'MCQ': 'What does a curved PPC line indicate?',
  'choices': 'a : Constant trade-off ratio | b : Diminishing marginal rate of transformation | c : No trade-offs between goods | d : Unrealistic production possibilities',
  'correct': 'b'},
 {'MCQ': 'How can technological advancements impact the PPC?',
  'choices': 'a : Shift the curve inwards | b : Decrease production possibilities | c : Shift the curve outwards for greater production possibil

In [383]:
quiz = pd.DataFrame(quiz_table_data)

In [384]:
quiz.to_csv("quiz.csv", index=False)